In [1]:
import pandas as pd
import sqlite3

In [7]:
sql_conn = sqlite3.connect('../data/checking-logs.sqlite')

In [8]:
query = """
SELECT
    CASE WHEN t.first_commit_ts < t.first_view_ts THEN 'before' ELSE 'after' END AS time,
    AVG(CAST((JulianDay(t.first_commit_ts) - JulianDay(d.deadlines, 'unixepoch')) * 24 AS INTEGER)) AS avg_diff
FROM test t
JOIN deadlines d
ON t.labname=d.labs
WHERE labname<>'project1' AND uid IN (SELECT uid
                                      FROM (SELECT t.uid,
                                                    CASE WHEN t.first_commit_ts < t.first_view_ts THEN 'before' ELSE 'after' END AS time
                                            FROM test t
                                            JOIN deadlines d
                                            ON t.labname = d.labs
                                            WHERE labname<>'project1')
                                      GROUP by uid
                                      HAVING COUNT(DISTINCT time)=2)
GROUP by time
"""
test_results = pd.io.sql.read_sql(query, sql_conn)
test_results

,time,avg_diff
0,after,-104.6000
1,before,-60.5625


In [9]:
query = '''
SELECT CASE WHEN c.first_commit_ts < c.first_view_ts THEN 'before' ELSE 'after' END AS time,
        AVG(CAST((JulianDay(c.first_commit_ts) - JulianDay(d.deadlines, 'unixepoch')) * 24 AS INTEGER)) AS avg_diff
FROM control c
JOIN deadlines d
ON c.labname = d.labs
WHERE labname <> 'project1'
AND uid IN (SELECT uid
            FROM (SELECT c.uid, CASE WHEN c.first_commit_ts < c.first_view_ts THEN 'before' ELSE 'after' END AS time
                  FROM control c
                  JOIN deadlines d
                  ON c.labname=d.labs
                  WHERE labname<>'project1')
            GROUP by uid
            HAVING COUNT(DISTINCT time)=2)
GROUP by time
'''
control_results = pd.io.sql.read_sql(query, sql_conn)
control_results

,time,avg_diff
0,after,-117.636364
1,before,-99.464286


In [10]:
sql_conn.close()